In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110690

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,45050.4712
2018-02-28,57670.7862
2018-03-31,49190.8076
2018-04-30,58365.5514
2018-05-31,66429.9521
2018-06-30,70163.5438
2018-07-31,73956.0732
2018-08-31,90050.1615
2018-09-30,85143.6079


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31     45050.4712
 2018-02-28     57670.7862
 2018-03-31     49190.8076
 2018-04-30     58365.5514
 2018-05-31     66429.9521
 2018-06-30     70163.5438
 2018-07-31     73956.0732
 2018-08-31     90050.1615
 2018-09-30     85143.6079
 2018-10-31     68814.5116
 2018-11-30     60315.4241
 2018-12-31     75957.8587
 2019-01-31     86563.4463
 2019-02-28     87754.3446
 2019-03-31     81855.6919
 2019-04-30     92095.8792
 2019-05-31     99567.8251
 2019-06-30    105996.3857
 2019-07-31    115024.3076
 2019-08-31     97716.6946
 2019-09-30     89719.7511
 2019-10-31     77781.8975
 2019-11-30     75468.2717
 2019-12-31     86960.9237
 2020-01-31     85443.3719
 2020-02-29     46776.1702
 2020-03-31     47747.9015
 2020-04-30     40208.6764
 2020-05-31     45309.5033
 2020-06-30     72558.9857
 2020-07-31     99021.4814
 2020-08-31     80771.4285
 2020-09-30     58291.1176
 2020-10-31     60250.9258
 2020-11-30     46886.1544
 2020-12-31     42846.2800

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.698838
p-value : 0.074240
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.201187
p-value : 0.000009
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=955.312, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=932.378, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=926.358, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=924.439, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=924.529, Time=0.06 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=925.965, Time=0.08 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=916.609, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=914.166, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=916.263, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=918.173, Time=0.07 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=920.516, Time=0.11 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=914.675, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=916.208, Time=0.07 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=918.081, Time=0.08 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=920.019, Time=0.16 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(1, 0, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,1))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 1)   Log Likelihood                -453.028
Method:                       css-mle   S.D. of innovations          11530.150
Date:                Sat, 11 Sep 2021   AIC                            914.056
Time:                        04:22:40   BIC                            921.007
Sample:                             0   HQIC                           916.604
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.856e+04   7075.617      9.690      0.000    5.47e+04    8.24e+04
ar.L1.y        0.6628      0.133      4.986      0.000       0.402       0.923
ma.L1.y        0.4066      0.162      2.509      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([69760.5510727 , 69357.13223505]), array([11530.15019845, 16881.2490974 ]), array([[ 47161.87194741,  92359.23019799],
       [ 36270.4919901 , 102443.77248   ]]))
